# LLM - Visual captioning

## Imagen imagetext
- https://cloud.google.com/vertex-ai/docs/generative-ai/image/image-captioning

In [1]:

import vertexai
from google.cloud import bigquery
from tools import *


PROJECT_ID = "ml-demo-384110"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")
client = bigquery.Client(project=PROJECT_ID)

%load_ext google.cloud.bigquery

In [2]:
# @title Parameters
LOCATION = "us-central1" #@param {type:"string"}
#PROJECT_NUMBER = 
#PROJECT_ID = "" #@param {type:"string"}

PROJECT_ID = !gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_ID)"

if len(PROJECT_ID) > 1:
    PROJECT_ID = PROJECT_ID[1]
else:
    PROJECT_ID = PROJECT_ID[0]
print(PROJECT_ID)

PROJECT_NUMBER = !gcloud projects list --filter="$(gcloud config get-value project)" --format="value(PROJECT_NUMBER)"
if len(PROJECT_NUMBER) > 1:
    PROJECT_NUMBER = PROJECT_NUMBER[1]
else:
    PROJECT_NUMBER = PROJECT_NUMBER[0]

print(PROJECT_NUMBER)

TOKEN_ENDPOINT_URI = 'https://www.googleapis.com/oauth2/v3/tokeninfo?access_token='

IMAGEGEN_MODEL = "imagegeneration@002" #models available: imagegeneration@001 and imagegeneration@002
ENDPOINT_URL = f'projects/{PROJECT_NUMBER}/locations/{LOCATION}/publishers/google/models/{IMAGEGEN_MODEL}'

IMAGE_TEXT_MODEL = "imagetext"
IMAGEN_ENDPOINT_URL = f'projects/{PROJECT_NUMBER}/locations/{LOCATION}/publishers/google/models/{IMAGE_TEXT_MODEL}'


ml-demo-384110
1008225662928


In [3]:
"""
Get access token
"""
gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get(TOKEN_ENDPOINT_URI + gcloud_token[0]).json()
print("Access Token is generated for ", gcloud_tokeninfo['email'])
ACCESS_TOKEN = gcloud_token[0]

#print(headers)

Access Token is generated for  admin@julienmiquel.altostrat.com


In [4]:
# @title Setup

import json
import requests
import time

from IPython import display
import base64

# @title image Generation Model
def generate_image(
    prompt: str,
    sampleImageSize: str,
    sampleCount: int,
    seed: int
    ):
    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
    }
    data = {"instances": [{"prompt": prompt}],
            "parameters": {"sampleImageSize": sampleImageSize,"sampleCount": sampleCount, "seed": seed}}

    response = requests.post(f'https://{LOCATION}-aiplatform.googleapis.com/v1/{ENDPOINT_URL}:predict', data=json.dumps(data), headers=headers)
    return response

###
# Visual Question Answering (VQA) to get image information
##
def visual_qa(
    prompt: str,
    sampleCount: int,
    encoded_str: str
    ):
    
    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
    }
    
    data = {
        "instances": [
            {
            "prompt": prompt,
            "image": {
                "bytesBase64Encoded": encoded_str
            }
            }
        ],
        "parameters": {
            "sampleCount": sampleCount
            }
        }

    response = requests.post(f'https://{LOCATION}-aiplatform.googleapis.com/v1/{IMAGEN_ENDPOINT_URL}:predict', 
        data=json.dumps(data), headers=headers)

    return response

###
#Visual captioning is available in the following languages:
# English (en)
# French (fr)
# German (de)
# Italian (it)
# Spanish (es)
##
def visual_captioning(
    sampleCount: int,
    language: str, 
    # English (en), French (fr), German (de), Italian (it), Spanish (es)
    encoded_str: str
    ):
    
    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}',
      'Content-Type': 'application/json; charset=UTF-8'
    }
    
    data = {
        "instances": [
            {
            "image": {
                "bytesBase64Encoded": encoded_str
                }
            }
        ],
        "parameters": {
            "sampleCount": sampleCount,
            "language": language
            }
        }

    response = requests.post(f'https://{LOCATION}-aiplatform.googleapis.com/v1/{IMAGEN_ENDPOINT_URL}:predict', 
        data=json.dumps(data), headers=headers)

    return response


In [5]:
from tools import *



base_image_path = "dataset/immo/images/_photos_realestate_2023-02_product-1303126-3.jpg"

bucket = "ml-demo-384110-bucket"
#str_img = read_b64_from_gcs(bucket, base_image_path)



vqa_text = print_text_response(    visual_captioning( 3, "en",str(read_b64_from_gcs(bucket, base_image_path), 'UTF-8')))
vqa_text2 = print_text_response(    visual_captioning( 3, "fr",str(read_b64_from_gcs(bucket, base_image_path), 'UTF-8')))




['a house with a lot of potted plants in front of it', 'a picture of a house with the letters iad on the bottom', 'a house with a lot of potted plants in front of it and a sign that says iad']
a house with a lot of potted plants in front of it
["l' arrière d' une maison avec des pots de fleurs à l' extérieur", "l' arrière d' une maison avec des pots de fleurs et des arbres", "l' arrière d' une maison avec des pots de fleurs à côté"]
l' arrière d' une maison avec des pots de fleurs à l' extérieur


In [34]:
vqa_text2

"l' arrière d' une maison avec des pots de fleurs à l' extérieur"

In [11]:
%%bigquery df
      SELECT uri, Id,
      FROM `immo_us_enriched.image_cloudvision_labels_1k` 


Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
df

,uri,attributes,Id
0,gs://ml-demo-384110-bucket/dataset/immo/images...,"Property, Furniture, Cabinetry, Comfort, Wood,...",ag060811-378090001
1,gs://ml-demo-384110-bucket/dataset/immo/images...,"Furniture, Property, Building, Window, Comfort...",ag060811-378090001


In [23]:
from ratelimit import limits, sleep_and_retry

from tools import *

@sleep_and_retry
@limits(calls=5, period=60)
def add_images_caption_to_df(df, language = "en", bucket_name = "ml-demo-384110-bucket"):

    df[f'caption_{language}' ] = df.uri.map(lambda uri_name:                                    
                                   
                                   print_text_response(visual_captioning( 1, language,str(read_b64_from_gcs(bucket, uri_name.replace(f"gs://{bucket_name}/", "")), 'UTF-8'))
                                  )
                                  )
    
    return df


In [24]:
from bq import save_bq



BQ_DATASET_NAME = 'immo_us' 
BQ_TABLE_NAME = 'vqa'
project_table = PROJECT_ID + "." +BQ_DATASET_NAME +"." + BQ_TABLE_NAME


#df_vqa = add_images_caption_to_df(df)
#save_bq(df_vqa, table_id=project_table, project_id=PROJECT_ID)


%pip install --upgrade pandas_gbq

In [25]:
def process(df):
    for id in df.Id :

        print(id)
        df_test1 = df[df["Id"] == id]
        
        df_vqa = add_images_caption_to_df(df_test1)

        
        df_vqa["update_time"] = update_time

        print(f"SAVE index chunk : {id}")
        save_bq(df_vqa, table_id=project_table, project_id=PROJECT_ID)

process(df)

ag060811-378090001
['a bedroom with a bed and a skylight and the word optimhome on the bottom']
a bedroom with a bed and a skylight and the word optimhome on the bottom
['a bedroom with a bed and a picture on the wall that says imhome.com']
a bedroom with a bed and a picture on the wall that says imhome.com
['a living room with a red couch and stairs leading to the kitchen']
a living room with a red couch and stairs leading to the kitchen
['a bedroom with the website optimhome.com on the bottom']
a bedroom with the website optimhome.com on the bottom
['a living room with a fireplace and bookshelves with the word com on the bottom']
a living room with a fireplace and bookshelves with the word com on the bottom
['a wooden staircase with optimhome.com written on the bottom']
a wooden staircase with optimhome.com written on the bottom
['a bathroom with the website optimhome.com on the bottom']
a bathroom with the website optimhome.com on the bottom
['a bathroom with the website optimhome.c

/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_67852/4080904126.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'caption_{language}' ] = df.uri.map(lambda uri_name:
100%|██████████| 1/1 [00:00<00:00, 5882.61it/s]


ag060811-378090001
['a bedroom with a bed and a skylight and the words optimhome on the bottom']
a bedroom with a bed and a skylight and the words optimhome on the bottom
['a bedroom with a bed and a picture on the wall that says imhome.com']
a bedroom with a bed and a picture on the wall that says imhome.com
['a living room with a red couch and stairs leading to the kitchen']
a living room with a red couch and stairs leading to the kitchen
['a bedroom with the website optimhome.com on the bottom']
a bedroom with the website optimhome.com on the bottom
['a living room with a fireplace and bookshelves with the word com on the bottom']
a living room with a fireplace and bookshelves with the word com on the bottom
['a staircase with optimhome.com written on the bottom']
a staircase with optimhome.com written on the bottom
['a bathroom with the website optimhome.com on the bottom']
a bathroom with the website optimhome.com on the bottom
['a bathroom with the website optimhome.com on the bo

/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_67852/4080904126.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'caption_{language}' ] = df.uri.map(lambda uri_name:
100%|██████████| 1/1 [00:00<00:00, 8830.11it/s]


ag060811-378090001
['a bedroom with a bed and a skylight and the words optimhome on the bottom']
a bedroom with a bed and a skylight and the words optimhome on the bottom
['a bedroom with a bed and a picture on the wall that says imhome.com']
a bedroom with a bed and a picture on the wall that says imhome.com
['a living room with a red couch and stairs leading to the kitchen']
a living room with a red couch and stairs leading to the kitchen
['a bedroom with the website optimhome.com on the bottom']
a bedroom with the website optimhome.com on the bottom
['a living room with a fireplace and bookshelves with the word com on the bottom']
a living room with a fireplace and bookshelves with the word com on the bottom
['a staircase with optimhome.com written on the bottom']
a staircase with optimhome.com written on the bottom
['a bathroom with the website optimhome.com on the bottom']
a bathroom with the website optimhome.com on the bottom
['a bathroom with the website optimhome.com on the bo

...

/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_67852/4080904126.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'caption_{language}' ] = df.uri.map(lambda uri_name:
100%|██████████| 1/1 [00:00<00:00, 10106.76it/s]


ag060811-383081582
['a poster of a man in a suit hangs above a wooden staircase']
a poster of a man in a suit hangs above a wooden staircase
['a bathroom with blue and white tiles and a bathtub']
a bathroom with blue and white tiles and a bathtub
['a bathroom with two sinks and a large mirror']
a bathroom with two sinks and a large mirror
['a bathroom with a white sink and a mirror']
a bathroom with a white sink and a mirror
['a bedroom with a bed and a trombone on the floor']
a bedroom with a bed and a trombone on the floor
['a bedroom with pink walls and a bed']
a bedroom with pink walls and a bed
["a bedroom with a bed and a shelf full of books including one that says ' sherlock holmes ' on it"]
a bedroom with a bed and a shelf full of books including one that says ' sherlock holmes ' on it
['a room with a desk and a computer on it']
a room with a desk and a computer on it
['a kitchen with a clock on the wall that shows the time as 10:10']
a kitchen with a clock on the wall that sho

/var/folders/2_/jl1djv6975g5c0pygxnl873w00pwkm/T/ipykernel_67852/4080904126.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'caption_{language}' ] = df.uri.map(lambda uri_name:
100%|██████████| 1/1 [00:00<00:00, 2987.40it/s]


ag060811-383081582
['a poster of a man in a suit hangs above a wooden staircase']
a poster of a man in a suit hangs above a wooden staircase
['a bathroom with blue and white tiles and a bathtub']
a bathroom with blue and white tiles and a bathtub
['a bathroom with two sinks and a large mirror']
a bathroom with two sinks and a large mirror
['a bathroom with a white sink and a mirror']
a bathroom with a white sink and a mirror
['a bedroom with a bed and a trombone on the floor']
a bedroom with a bed and a trombone on the floor
['a bedroom with pink walls and a bed']
a bedroom with pink walls and a bed
["a bedroom with a bed and a shelf full of books including one that says ' sherlock holmes ' on it"]
a bedroom with a bed and a shelf full of books including one that says ' sherlock holmes ' on it
['a room with a desk and a computer on it']
a room with a desk and a computer on it
['a kitchen with a clock on the wall that shows the time as 10:10']
a kitchen with a clock on the wall that sho

KeyboardInterrupt: 

In [1]:
BQ_DATASET_NAME = 'immo_us' 
BQ_TABLE_NAME = 'vqa'
project_table = PROJECT_ID + "." +BQ_DATASET_NAME +"." + BQ_TABLE_NAME

NameError: name 'PROJECT_ID' is not defined

In [12]:
%%bigquery df
      SELECT uri, Id,
      FROM `immo_us_enriched.image_cloudvision_labels_1k` 


,uri,Id
0,gs://ml-demo-384110-bucket/dataset/immo/images...,ag060811-378090001
1,gs://ml-demo-384110-bucket/dataset/immo/images...,ag060811-378090001
2,gs://ml-demo-384110-bucket/dataset/immo/images...,ag060811-378090001
3,gs://ml-demo-384110-bucket/dataset/immo/images...,ag060811-378090001
